In [ ]:
import pandas as pd

In [19]:
def calculer_scores(df,template, poids):
    """
    Calcule les scores pour un DataFrame contenant des colonnes discrètes et continues.

    Args:
    df (DataFrame): DataFrame contenant les données.
    poids_continu (float): Poids pour les colonnes continues.
    poids_discret (float): Poids pour les colonnes discrètes.

    Returns:
    DataFrame: DataFrame avec les scores calculés et la colonne 'score global'.
    """
    # Cloner le DataFrame d'origine
    df_scores = df.copy()

    # Calculer les scores pour chaque colonne
    for colonne in poids:
        if df[colonne].dtype == 'object' or df[colonne].dtype =='int64':
            # Si la colonne est discrète
            df_scores[f'score_{colonne}'] = df[colonne].apply(score_proximite_discret, args=(template[colonne],)) * poids[colonne]
        else:
            # Si la colonne est continue
            # Calculer la distance maximale pour la colonne continue
            max_distance = max(abs(df[colonne] - template[colonne]))

            df_scores[f'score_{colonne}'] = df[colonne].apply(score_proximite, args=(template[colonne],max_distance)) * poids[colonne]

    # Calculer le score global
    df_scores['score_global'] = df_scores[[col for col in df_scores.columns if col.startswith('score_')]].sum(axis=1)

    # Retourner la ligne avec le plus grand score global
    ligne_max_score = df_scores.loc[df_scores['score_global'].idxmax()]

    return df_scores, ligne_max_score

def score_proximite_discret(valeur, template):
    """
    Calcule le score de proximité d'une valeur par rapport au template de manière discrète.

    Args:
    valeur (str or int): Valeur discrète.
    template (str or int): Valeur template.

    Returns:
    float: Score de proximité (1 si la valeur est exactement la même que le template, sinon 0).
    """
    score = 1 if valeur == template else 0
    return score

def score_proximite(valeur, template, max_distance):
    """
    Calcule le score de proximité d'une valeur par rapport au template.

    Args:
    valeur (float): Valeur numérique.
    template (float): Valeur template.

    Returns:
    float: Score de proximité.
    """
    distance = abs(valeur - template)
    score = 1 - distance / max_distance
    return score

def get_settings(path = "settings.xlsx"): 

    df = pd.read_excel(path)

    dictionnaire_colonnes = df.iloc[0].to_dict()
    if 'id' in dictionnaire_colonnes:
        dictionnaire_colonnes.pop('id')

    somme_valeurs = sum(dictionnaire_colonnes.values())

    return {cle: valeur / somme_valeurs for cle, valeur in dictionnaire_colonnes.items()}


In [24]:
# Exemple d'utilisation
df = pd.read_excel('appartements.xlsx')
poids = get_settings()
template = df.iloc[0].to_dict()

# Calculer les scores
df_scores, ligne_max_score = calculer_scores(df, template, poids)

print("DataFrame avec les scores calculés :\n", df_scores)
print("\nLigne avec le plus grand score global :\n", ligne_max_score['id'])

DataFrame avec les scores calculés :
      id type_appart  nb_murs  nb_murs-1m  surface  pos_f1  pos_f2  pos_f3  \
0  T2-0          T2        6           0    39.55       4       5       0   
1  T2-1          T2        6           0    33.94       4       0       0   
2  T2-2          T2        4           0    38.17       3       0       0   
3  T2-3          T2        4           0    51.00       1       2       3   
4  T2-4          T2        6           0    46.00       3       5       0   

   pos_f4  pos_f5  ...  score_nb_murs-1m  score_surface  score_pos_f1  \
0       0       0  ...          0.052632       0.078947      0.105263   
1       0       0  ...          0.052632       0.040267      0.105263   
2       0       0  ...          0.052632       0.069432      0.000000   
3       4       0  ...          0.052632       0.000000      0.000000   
4       0       0  ...          0.052632       0.034475      0.000000   

   score_pos_f2  score_pos_f3  score_pos_f4  score_pos_f5  s